In [92]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [93]:
vocab_size = 1909
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print len(d)
x, y = d[10:20], d[11:21]
print [np.argmax(j) for j in x]
print [np.argmax(j) for j in y]

[u'in', u'the', u'beginning', u'god', u'created', u'the', u'heaven', u'and', u'the', u'earth', u'.', u'and', u'the', u'earth', u'was', u'without', u'form', u',', u'and', u'void', u';', u'and', u'darkness', u'was', u'upon', u'the', u'face', u'of', u'the', u'deep', u'.', u'and', u'the', u'spirit', u'of', u'god', u'moved', u'upon', u'the', u'face']
1909
29602
[5, 1, 3, 45, 24, 466, 1421, 2, 1, 1131]
[1, 3, 45, 24, 466, 1421, 2, 1, 1131, 9]


In [104]:
# hyper-parameters
input_size = output_size = vocab_size
iterations = 10000
hidden_layer = 128
inp_out_size = vocab_size
learning_rate = 0.1
num_steps = 26

In [95]:
tf.reset_default_graph()
initializer = tf.random_normal_initializer(mean=0, stddev=0.001, dtype=tf.float32)
Wxh = tf.get_variable('Wxh', shape=[input_size, hidden_layer], initializer=initializer)
Whh = tf.get_variable('Whh', shape=[hidden_layer, hidden_layer], initializer=initializer)
Why = tf.get_variable('Why',shape=[hidden_layer, output_size], initializer=initializer)
by = tf.get_variable('by', shape=[output_size], initializer=initializer)
# weights associated with update gate
Wxz = tf.get_variable('Wxz', shape=[input_size, hidden_layer], initializer=initializer)
Whz = tf.get_variable('Whz', shape=[hidden_layer, hidden_layer], initializer=initializer)
# weights associated with the reset gate
Wxr = tf.get_variable('Wxr', shape=[input_size, hidden_layer], initializer=initializer)
Whr = tf.get_variable('Whr', shape=[hidden_layer, hidden_layer], initializer=initializer)

In [96]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, Wxz) + tf.matmul(p, Whz))    # update gate
    r = tf.nn.sigmoid(tf.matmul(i, Wxr) + tf.matmul(p, Whr))    # reset gate
    h_ = tf.nn.tanh(tf.matmul(i, Wxh) + tf.matmul(tf.mul(p, r), Whh))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [hidden_layer])

In [97]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.AdagradOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -4, 4), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

# optimize_op = optimizer.minimize(loss)

In [98]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print ' '.join(gen)

In [99]:
init = tf.initialize_all_variables()
saver = tf.train.Saver()
sess = tf.Session()
ix = 0
smooth_loss = -np.log(1.0 / vocab_size) * num_steps

In [100]:
# if already trained previously, just restore
to_restore = False

if to_restore:
    saver.restore(sess, 'model.ckpt')
else:
    sess.run(init)

In [ ]:
print epochs
h = np.zeros(hidden_layer)
for i in range(iterations):
#     x, y = data.sample(d, num_steps)
    if ix + num_steps >= len(d):
        ix = 0
        print('one epoch complete')
    h = np.zeros(hidden_layer)
    x, y = d[ix : ix + num_steps], d[ix + 1 : ix + num_steps + 1]   
    l, o, _ = sess.run([loss, outputs, optimize_op], {a: x, b: y, initial: h}) 
    smooth_loss = smooth_loss * 0.999 + l * 0.001
    if i % 1000 == 0:
        print('iteration {0}, loss = {1}'.format(i, l))
    ix += num_steps

In [ ]:
# saving model
# saver.save(sess, 'model.ckpt')

In [107]:
generate(sess, 500)

began to be his wife ; then she went down to her , and she conceived , and bare jacob a son son , and said , she hath taken away my blessing , i have made him thee . and he said , i am thy made , my lord hath blessed me , who hath also born me thee thy brother , that he may bless thee , and she also and her . and she conceived again , and bare a son , and said , i will surely give the god of my father give it , and to my master , when thou shalt have seen thy face , and thy cattle , and my soul shall live : and i will make thee my son , and let thee go : and we have not touched thee . and if we will bless thee , and let me drink also , and we will dwell with thee , and i will go thee my : and god saw that thou shalt be now unto me , and let my brethren , let us go : now i am speckled , and i will make thee also : let us make our daughters . and the daughters of my father 's serve with thee . and they said unto them , we be the daughter of my father , and hath given jacob 's and drink ,

In [ ]:
# sess.run(Whh)